In [ ]:
%load_ext autoreload
%autoreload 2

import json
import pickle
from chexmix.datasources import mesh, taxonomy

from chexmix.graph import BioGraph,TaxonomyGraph, MeSHGraph, PubTatorGraph, PubMedGraph, TaxParentType, NodeType, EdgeType

from chexmix.utils import remove_symbols

In [ ]:
## KPEB data load
with open('../data/KPEB/KPEB_name_taxID.json', 'r') as f:
    kpeb_data = json.load(f)
kpeb_data = {int(k): v for k,v in kpeb_data.items()}

##viridiplantae load
with open('../data/viridiplanae.txt', 'rb') as f:
    viridiplantae = pickle.load(f)

In [ ]:
mesh_table = mesh.load_mesh()

In [ ]:
tax_table = taxonomy.load_taxonomy()

In [ ]:
tax_table['TAXO:63221']

In [ ]:
path = '../data/network/'

In [ ]:
keyword = 'podophyllum peltatum'

In [ ]:
parent_node_type = TaxParentType.Genus # Genus or Family
input_names = ['podophyllum peltatum']

In [ ]:
biographs_of_keyword = []
# create keyword graph
for input_name in input_names:    
    file_name = path + remove_symbols(input_name)
    pubmed_graph = PubMedGraph.from_keyword(input_name)
    article_ids = pubmed_graph.get_article_ids()
    pubtator_graph = PubTatorGraph.from_article_ids(article_ids)
    tax_raw_graph = TaxonomyGraph.from_table(tax_table)
    mesh_raw_graph = MeSHGraph.from_table(mesh_table)
    bioentities = pubtator_graph.get_bioentities(['TAXO'])

    tax_graph = tax_raw_graph.subgraph_from_pubtator_bioentities(tax_table,parent_node_type, pubtator_graph.get_bioentities(['TAXO']), viridiplantae, 'KPEB', kpeb_data)
    mesh_graph = mesh_raw_graph.subgraph_from_pubtator_bioentities(pubtator_graph.get_bioentities(['MSHD','MSHC']))

    biograph_of_keyword = BioGraph()
    biograph_of_keyword.add_edges_from([(input_name, root_node) for root_node in tax_graph.find_roots()])
    mesh_entities_graph = mesh_graph.intersection(pubtator_graph)
    biograph_of_keyword.add_edges_from([(input_name, appeared_node) for appeared_node in mesh_entities_graph.nodes()])
    biograph_of_keyword.nodes[input_name]['type'] = "Keyword"
    biograph_of_keyword.inherit_attr_from(tax_graph.union(mesh_graph))
    # nx.write_graphml(biograph_of_keyword, f'{file_name}.graphml')
    biographs_of_keyword.append(biograph_of_keyword)